In [1]:
# !pip install wilds

     |████████████████████████████████| 126 kB 5.3 MB/s 
     |████████████████████████████████| 503 kB 44.7 MB/s 
     |████████████████████████████████| 38.1 MB 1.5 MB/s 
     |████████████████████████████████| 78 kB 6.2 MB/s 
     |████████████████████████████████| 4.3 MB 34.2 MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=0f14ad6f0ce8c83a43047049f8dfca4142c36e3392727ac1563f341c22d9313e
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built littleutils
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: pytz
    Found existing installation: pytz 2018.9
    Uninstalling pytz-2018.9:
      Successfully uninstalled pytz-2018.9
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Success

In [1]:
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
import torchvision.models as models
from wilds.common.data_loaders import get_eval_loader
import torch
from torch import nn
from tqdm import tqdm

In [2]:
# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="camelyon17", download=True)


You can also download the dataset manually at https://wilds.stanford.edu/downloads.


  0%|          | 0/10658709504 [00:00<?, ?Byte/s]

Extracting data/camelyon17_v1.0/archive.tar.gz to data/camelyon17_v1.0

It took 55.76 minutes to download and uncompress the dataset.



In [3]:
BATCH_SIZE = 32
FRACTION = 0.33

# Get the training set
train_data = dataset.get_subset(
    "train",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(train_data)) #302436 initially
# Prepare the standard data loader
train_loader = get_train_loader("standard", train_data, batch_size=BATCH_SIZE)

"""
# (Optional) Load unlabeled data
dataset = get_dataset(dataset="camelyon17", download=True, unlabeled=True)
unlabeled_data = dataset.get_subset(
    "test_unlabeled",
    transform=transforms.Compose(
        [transforms.Resize((448, 448)), transforms.ToTensor()]
    ),
)
unlabeled_loader = get_train_loader("standard", unlabeled_data, batch_size=16)
"""
"""
# Train loop
for labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):
    x, y, metadata = labeled_batch
    unlabeled_x, unlabeled_metadata = unlabeled_batch
    ...
"""

99804


'\n# Train loop\nfor labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):\n    x, y, metadata = labeled_batch\n    unlabeled_x, unlabeled_metadata = unlabeled_batch\n    ...\n'

In [4]:
# Get the test set
id_val_data = dataset.get_subset(
    "id_val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(id_val_data))

# Prepare the evaluation data loader
id_val_loader = get_eval_loader("standard", id_val_data, batch_size=BATCH_SIZE)


11075


In [77]:
# Get the test set
val_data = dataset.get_subset(
    "val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(val_data))

# Prepare the evaluation data loader
val_loader = get_eval_loader("standard", val_data, batch_size=BATCH_SIZE)

11518


In [6]:
# Get the test set
test_data = dataset.get_subset(
    "test",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(test_data))

# Prepare the evaluation data loader
test_loader = get_eval_loader("standard", test_data, batch_size=BATCH_SIZE)

28068


In [7]:

# load the ResNet-18 model, with weights pretrained on ImageNet
resnet18_pretrained = models.resnet18(pretrained=True)

num_params = 0
print("Model's parameters: ")
for n, p in resnet18_pretrained.named_parameters():
    print('\t', n, ': ', p.size())
    num_params += p.numel()
print("Number of model parameters: ", num_params)

"""
formula [(W−K+2P)/S]+1.

W is the input volume
K is the Kernel size
P is the padding
S is the stride

"""

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Model's parameters: 
	 conv1.weight :  torch.Size([64, 3, 7, 7])
	 bn1.weight :  torch.Size([64])
	 bn1.bias :  torch.Size([64])
	 layer1.0.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn1.weight :  torch.Size([64])
	 layer1.0.bn1.bias :  torch.Size([64])
	 layer1.0.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn2.weight :  torch.Size([64])
	 layer1.0.bn2.bias :  torch.Size([64])
	 layer1.1.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn1.weight :  torch.Size([64])
	 layer1.1.bn1.bias :  torch.Size([64])
	 layer1.1.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn2.weight :  torch.Size([64])
	 layer1.1.bn2.bias :  torch.Size([64])
	 layer2.0.conv1.weight :  torch.Size([128, 64, 3, 3])
	 layer2.0.bn1.weight :  torch.Size([128])
	 layer2.0.bn1.bias :  torch.Size([128])
	 layer2.0.conv2.weight :  torch.Size([128, 128, 3, 3])
	 layer2.0.bn2.weight :  torch.Size([128])
	 layer2.0.bn2.bias :  torch.Size([128])
	 layer2.0.downsample.0.weight :  torch.Size([12

'\nformula [(W−K+2P)/S]+1.\n\nW is the input volume\nK is the Kernel size\nP is the padding\nS is the stride\n\n'

In [10]:
# function counting the number of parameters and the number of trainable parameters of a model
# optionally, it will also display the layers
def check_model_parameters(model, display_layers=False):
  num_params = 0
  num_trainable_params = 0
  if display_layers==True:
    print("Model's parameters: ")
  for n, p in model.named_parameters():
      if display_layers == True:
        print('\t', n, ': ', p.size())
      num_params += p.numel()
      if p.requires_grad:
        num_trainable_params += p.numel()
  print("Number of model parameters: ", num_params)
  print("Number of trainable parameters: ", num_trainable_params)

In [ ]:
# freeze the model parameters

# check the number of parameters and the number of trainable parameters
check_model_parameters(resnet18_pretrained, display_layers=False)

# freeze all the layers
# for param in resnet18_pretrained.parameters():
  # param.requires_grad = False

# check the number of parameters and the number of trainable parameters
check_model_parameters(resnet18_pretrained, display_layers=False)

Number of model parameters:  11689512
Number of trainable parameters:  11689512
Number of model parameters:  11689512
Number of trainable parameters:  11689512


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 5

In [11]:

resnet18_pretrained.fc = nn.Linear(in_features=512, out_features=2, bias=True)

check_model_parameters(resnet18_pretrained, display_layers=True)

Model's parameters: 
	 conv1.weight :  torch.Size([64, 3, 7, 7])
	 bn1.weight :  torch.Size([64])
	 bn1.bias :  torch.Size([64])
	 layer1.0.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn1.weight :  torch.Size([64])
	 layer1.0.bn1.bias :  torch.Size([64])
	 layer1.0.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn2.weight :  torch.Size([64])
	 layer1.0.bn2.bias :  torch.Size([64])
	 layer1.1.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn1.weight :  torch.Size([64])
	 layer1.1.bn1.bias :  torch.Size([64])
	 layer1.1.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn2.weight :  torch.Size([64])
	 layer1.1.bn2.bias :  torch.Size([64])
	 layer2.0.conv1.weight :  torch.Size([128, 64, 3, 3])
	 layer2.0.bn1.weight :  torch.Size([128])
	 layer2.0.bn1.bias :  torch.Size([128])
	 layer2.0.conv2.weight :  torch.Size([128, 128, 3, 3])
	 layer2.0.bn2.weight :  torch.Size([128])
	 layer2.0.bn2.bias :  torch.Size([128])
	 layer2.0.downsample.0.weight :  torch.Size([12

In [ ]:
def train_epoch(model, train_dataloader, loss_crt, optimizer, device):
    """
    model: Model object
    train_dataloader: DataLoader over the training dataset
    loss_crt: loss function object
    optimizer: Optimizer object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch training loss, which is an average over the individual batch
       losses
    """
    model.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    num_batches = len(train_dataloader)
    for batch_idx, batch in tqdm(enumerate(train_dataloader)):
        # shape: batch_size x 1 x 28 x 28, batch_size x 1
        # print("Train")
        # print(batch)
        batch_img, batch_labels, _ = batch
        # move data to GPU
        batch_img = batch_img.to(device)
        batch_labels = batch_labels.to(device)

        # initialize as zeros all the gradients of the model
        model.zero_grad()

        # get predictions from the FORWARD pass
        # shape: batch_size x 10
        output = model(batch_img)

        loss = loss_crt(output, batch_labels.squeeze())
        loss_scalar = loss.item()

        # BACKPROPAGATE the gradients
        loss.backward()
        # use the gradients to OPTIMISE the model
        optimizer.step()

        epoch_loss += loss_scalar

        pred = output.argmax(dim=1, keepdim=True)
        epoch_accuracy += pred.eq(batch_labels.view_as(pred)).float().mean().item()

    epoch_loss = epoch_loss/num_batches
    epoch_accuracy = 100. * epoch_accuracy/num_batches
    return epoch_loss, epoch_accuracy

def eval_epoch(model, val_dataloader, loss_crt, device):
    """
    model: Model object
    val_dataloader: DataLoader over the validation dataset
    loss_crt: loss function object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch validation loss, which is an average over the individual batch
       losses
    """
    model.eval()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    num_batches = len(val_dataloader)
    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(val_dataloader)):
            # print("Eval")
            # print(batch)
            # shape: batch_size x 3 x 28 x 28, batch_size x 1
            batch_img, batch_labels, _ = batch
            current_batch_size = batch_img.size(0)

            # move data to GPU
            batch_img = batch_img.to(device)
            batch_labels = batch_labels.to(device)

            # batch_size x 10
            output = model(batch_img)

            loss = loss_crt(output, batch_labels.squeeze())
            loss_scalar = loss.item()

            epoch_loss += loss_scalar

            pred = output.argmax(dim=1, keepdim=True)
            epoch_accuracy += pred.eq(batch_labels.view_as(pred)).float().mean().item()

    epoch_loss = epoch_loss/num_batches
    epoch_accuracy = 100. * epoch_accuracy/num_batches
    return epoch_loss, epoch_accuracy

In [12]:
resnet18_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(resnet18_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# evaluate the initial model
# val_loss, al_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
# print('Validation performance before finetuning -- loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


In [ ]:

# finetune the model
train_losses = []
train_accuracies = []
id_val_losses = []
id_val_accuracies = []
for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(resnet18_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))

In [ ]:
def run_eval_ood(model, loader, loss_criterion, device, eval_type):
  losses = []
  accuracies = []

  loss, accuracy = eval_epoch(model, loader, loss_criterion, device)
  losses.append(loss)
  accuracies.append(accuracy)
  print(eval_type + ' loss: %10.8f, accuracy: %10.8f'%(loss, accuracy))

In [ ]:
run_eval_ood(resnet18_pretrained, val_loader, loss_criterion, device, "val")

360it [00:43,  8.24it/s]

val loss: 0.46796423, accuracy: 87.63715279


In [ ]:
run_eval_ood(resnet18_pretrained, test_loader, loss_criterion, device, "test")

878it [01:48,  8.10it/s]

test loss: 1.12270584, accuracy: 77.62314920


In [ ]:
densenet121_pretrained = models.densenet121(pretrained=True)

# for param in densenet121_pretrained.parameters():
  # param.requires_grad = False

check_model_parameters(densenet121_pretrained, display_layers=False)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

Number of model parameters:  7978856
Number of trainable parameters:  7978856


In [ ]:

densenet121_pretrained.classifier = nn.Linear(in_features=1024, out_features=2, bias=True)
check_model_parameters(densenet121_pretrained, display_layers=False)

Number of model parameters:  6955906
Number of trainable parameters:  6955906


In [ ]:
densenet121_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(densenet121_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# evaluate the initial model
# val_loss, al_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
# print('Validation performance before finetuning -- loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))

# finetune the model
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
id_val_losses = []
id_val_accuracies = []

for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(densenet121_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(densenet121_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


3119it [15:59,  3.25it/s]
347it [00:48,  7.20it/s]



Epoch 1
train loss: 0.08947193, accuracy: 96.96717698
id_val loss: 0.04903783, accuracy: 98.27089337


3119it [15:52,  3.28it/s]
347it [00:47,  7.36it/s]



Epoch 2
train loss: 0.04349755, accuracy: 98.54520680
id_val loss: 0.03753684, accuracy: 98.72118156


3119it [15:50,  3.28it/s]
347it [00:47,  7.33it/s]



Epoch 3
train loss: 0.03116413, accuracy: 98.91276966
id_val loss: 0.03892054, accuracy: 98.68515850


3119it [15:50,  3.28it/s]
347it [00:46,  7.48it/s]



Epoch 4
train loss: 0.02413429, accuracy: 99.13834562
id_val loss: 0.04475565, accuracy: 98.57708934


3119it [15:48,  3.29it/s]
347it [00:47,  7.32it/s]


Epoch 5
train loss: 0.02237113, accuracy: 99.20547451
id_val loss: 0.04431844, accuracy: 98.48703170


In [ ]:
run_eval_ood(densenet121_pretrained, val_loader, loss_criterion, device, "val")

360it [00:49,  7.26it/s]

val loss: 0.61660476, accuracy: 85.78125000


In [ ]:
run_eval_ood(densenet121_pretrained, test_loader, loss_criterion, device, "test")

878it [02:00,  7.29it/s]

test loss: 0.91136280, accuracy: 79.53445330


In [ ]:
torch.save(resnet18_pretrained.state_dict(), "resnet18_pretrained.pt")

In [ ]:
torch.save(densenet121_pretrained.state_dict(), "densenet121_pretrained.pt")

In [ ]:
resnetxt50_pretrained = models.resnext50_32x4d(pretrained=True)

# freeze all the layers
# for param in resnetxt50_pretrained.parameters():
  # param.requires_grad = False

resnetxt50_pretrained.fc = nn.Linear(in_features=2048, out_features=2, bias=True)

resnetxt50_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(resnetxt50_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()


# finetune the model
train_losses = []
train_accuracies = []
id_val_losses = []
id_val_accuracies = []
for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(resnetxt50_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(resnetxt50_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

3119it [17:44,  2.93it/s]
347it [00:43,  7.91it/s]



Epoch 1
train loss: 0.08606202, accuracy: 97.03115696
id_val loss: 0.05096113, accuracy: 98.17182997


3119it [17:43,  2.93it/s]
347it [00:43,  7.97it/s]


...


Epoch 3
train loss: 0.02918555, accuracy: 98.98104360
id_val loss: 0.03763097, accuracy: 98.68515850


3119it [17:42,  2.93it/s]
347it [00:44,  7.88it/s]



Epoch 4
train loss: 0.02205006, accuracy: 99.21048413
id_val loss: 0.03875422, accuracy: 98.63112392


3119it [17:44,  2.93it/s]
347it [00:43,  7.97it/s]


Epoch 5
train loss: 0.01629151, accuracy: 99.41788233
id_val loss: 0.04178602, accuracy: 98.77521614


In [ ]:
run_eval_ood(resnetxt50_pretrained, val_loader, loss_criterion, device, "val")

360it [00:45,  7.89it/s]

val loss: 0.40985703, accuracy: 87.87210650


In [ ]:
run_eval_ood(resnetxt50_pretrained, test_loader, loss_criterion, device, "test")

878it [01:51,  7.87it/s]

test loss: 1.72625949, accuracy: 72.21668565


In [ ]:
torch.save(densenet121_pretrained.state_dict(), "resnetxt50_pretrained.pt")

In [13]:
resnet18_pretrained.load_state_dict(torch.load("resnet18_pretrained_all_grad.pt"))

<All keys matched successfully>

In [90]:
"""
Rupere cap de clasificare si calculare distanta l2 si cos medie intre ood si iid
"""

# putem sa folosim labels atunci cand facem distanta? (i.e. distanta intra-clasa sau inter-clasa)

def calc_dist_iid(model, dataloader, metric="l2"):
    modules = list(model.children())[:-1]
    # print(modules)
    model = nn.Sequential(*modules)
    average_l2_iid = [0, 0]
    count_labels = [0, 0]
    model.eval()
    num_batches = len(dataloader)
    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(dataloader)):
            batch_img, batch_labels, _ = batch

            batch_img = batch_img.to(device)
            batch_labels = batch_labels.to(device)

            output = model(batch_img)
            output = output.reshape(batch_img.shape[0], -1)
             #if batch_idx == 0:
                # print(output.shape)

            for idx_img_1 in range(batch_img.shape[0] - 1):
                for idx_img_2 in range(idx_img_1 + 1, batch_img.shape[0]):

                    if batch_labels[idx_img_1] == batch_labels[idx_img_2]:
                        if metric == "l2":
                            dist = torch.nn.functional.pairwise_distance(output[idx_img_1], output[idx_img_2])
                        elif metric == "cos":
                            cos = nn.CosineSimilarity(dim=1, eps=1e-6)
                            dist = cos(output[idx_img_1].reshape(1, -1), output[idx_img_2].reshape(1, -1))
                        # dist = (output[idx_img_1] - output[idx_img_2]).pow(2).sum(0).sqrt()
                        average_l2_iid[batch_labels[idx_img_1]] += dist.item()
                        count_labels[batch_labels[idx_img_1]] += 1
                        # mean +- std?

    average_l2_iid[0] /= count_labels[0]
    average_l2_iid[1] /= count_labels[1]

    return average_l2_iid

def calc_dist(model, dataloader, x, y, metric="l2"): # x-> image to be decided if iid or ood, y-> label
    modules = list(model.children())[:-1]
    model = nn.Sequential(*modules)
    average_l2 = 0
    count_labels = 0
    model.eval()
    num_batches = len(dataloader)
    # x, y, _ = img
    print(y)
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
        x_shape = list(x.shape)
        x = x.reshape(1, x_shape[0], x_shape[1], x_shape[2])

        output_x = model(x)
        output_x = output_x.reshape(1, -1)
        for batch_idx, batch in tqdm(enumerate(dataloader)):
            batch_img, batch_labels, _ = batch

            batch_img = batch_img.to(device)
            batch_labels = batch_labels.to(device)

            output = model(batch_img)
            output = output.reshape(batch_img.shape[0], -1)
            # if batch_idx == 0:
                # print(output.shape)

            for idx_img_1 in range(batch_img.shape[0]):

                if batch_labels[idx_img_1] == y:

                    if metric == "l2":
                        dist = torch.nn.functional.pairwise_distance(output[idx_img_1], output_x)
                    elif metric == "cos":
                        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
                        dist = cos(output[idx_img_1].reshape(1, -1), output_x.reshape(1, -1))
                    average_l2 += dist.item()
                    count_labels += 1

    return average_l2 / count_labels

In [35]:
dist_iid = calc_dist_iid(resnet18_pretrained, id_val_loader)
print(dist_iid)

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False), BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False), Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bia

1it [00:00,  5.47it/s]

torch.Size([32, 512])


347it [01:02,  5.52it/s]

[8.019813598933185, 49.94453152325986]


In [89]:
dist_iid_cos = calc_dist_iid(resnet18_pretrained, id_val_loader, "cos")
print(dist_iid_cos)

347it [01:27,  3.99it/s]

[0.7907370429616276, 0.7607978926063546]


In [79]:
batch_data, batch_labels, _ = next(iter(val_loader))
for idx in range(len(batch_labels)):
    print(calc_dist(resnet18_pretrained, id_val_loader, batch_data[idx], batch_labels[idx]))
for batch_idx, batch in tqdm(enumerate(val_loader)):
    if batch_idx > 200:
        batch_data, batch_labels, _ = batch
        break
for idx in range(len(batch_labels)):
    print(calc_dist(resnet18_pretrained, id_val_loader, batch_data[idx], batch_labels[idx]))

tensor(1)


2it [00:00, 12.26it/s]

torch.Size([32, 512])


347it [00:29, 11.83it/s]


48.38585352512889
tensor(1)


...

8.6255110665727
tensor(0)


2it [00:00, 11.82it/s]

torch.Size([32, 512])


347it [00:28, 12.25it/s]

9.903116635410049


In [91]:
batch_data, batch_labels, _ = next(iter(val_loader))
for idx in range(len(batch_labels)):
    print(calc_dist(resnet18_pretrained, id_val_loader, batch_data[idx], batch_labels[idx], "cos"))
for batch_idx, batch in tqdm(enumerate(val_loader)):
    if batch_idx > 200:
        batch_data, batch_labels, _ = batch
        break
for idx in range(len(batch_labels)):
    print(calc_dist(resnet18_pretrained, id_val_loader, batch_data[idx], batch_labels[idx], "cos"))

tensor(1)


347it [00:28, 12.05it/s]


0.186843495709266
tensor(1)


347it [00:28, 12.10it/s]


0.16675274863717562
tensor(1)


...

0.795344392048855
tensor(0)


347it [00:28, 12.28it/s]


0.64924507459505
tensor(0)


347it [00:28, 12.32it/s]

0.8110449216836099
